In [80]:
import json
import requests
from api_keys import gkey
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
from citipy import citipy
from scipy.stats import linregress
import csv
import gmaps
import os

In [17]:
weather_pd = pd.read_csv("../WeatherPy/weather_500_list.csv")
weather_pd['Humidity'].max()

100

In [18]:
# Configure gmaps with API key
gmaps.configure(api_key=gkey)

# Store 'Lat' and 'Lng' into  locations 
locations = weather_pd[["Latitude", "Longtitude"]].astype(float)

# Convert Poverty Rate to float and store


Humidity_float = weather_pd["Humidity"].astype(float)  

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=Humidity_float, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [19]:
weather_pd.head()

,Country,City,Longtitude,Latitude,Temperature,Cloudiness,Humidity,Wind Speed
0,NaN,Lüderitz,15.1594,-26.6481,295.31,54,50,3.62
1,CL,Punta Arenas,-70.9167,-53.1500,276.21,20,81,8.75
2,JP,Tanabe,135.3667,33.7333,297.00,75,80,0.45
3,TO,Vaini,-175.2000,-21.2000,296.24,75,94,3.09
4,RU,Provideniya,-173.3000,64.3833,275.30,100,98,3.58


In [94]:
# my condition: temperature between 60-70/ wind speed less than 3/cloudiness less than 10
greatest_condition_weather = weather_pd.loc[((weather_pd["Temperature"] > 270)&(weather_pd["Temperature"] < 300)&(weather_pd["Wind Speed"] < 4)&(weather_pd["Cloudiness"] <30))]
final_df = greatest_condition_weather.reset_index()
final_df = final_df.drop(columns = ['index'])
final_df

,Country,City,Longtitude,Latitude,Temperature,Cloudiness,Humidity,Wind Speed
0,BR,São Marcos,-51.0681,-28.9711,283.23,0,88,1.42
1,BR,Canguçu,-52.6756,-31.3950,282.79,0,82,1.49
2,US,Laguna,-121.4238,38.4210,290.57,1,88,0.89
3,US,Saint George,-113.5841,37.1041,297.31,1,16,2.24
4,CK,Avarua,-159.7750,-21.2078,296.18,20,64,3.60
...,...,...,...,...,...,...,...,...
89,BR,Tapes,-51.3958,-30.6733,285.98,0,82,2.68
90,PT,Praia da Vitória,-27.0667,38.7333,294.29,20,83,1.99
91,FR,Arrondissement du Havre,0.3333,49.5833,299.08,9,50,1.75
92,US,Davidson,-86.7833,36.1501,295.00,1,86,1.54


In [95]:
#hotel_list = []
#locations = final_df[["Lat", "Lng"]]
for index,row in final_df.iterrows():
    #airport_df[["Lat", "Lng"]]
    lat = row['Latitude']
    lng = row['Longtitude']
# set up a parameters dictionary
    params = {"location": f"{lat},{lng}",
              "radius": 5000,
              "type": "lodging",
              "key": gkey
             }

# base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
    response = requests.get(base_url, params=params)
    places_data = response.json()
    try:
        hotel_name = places_data['results'][0]['name']
        final_df.loc[index, "Hotel"] = hotel_name
    except (KeyError, IndexError):
        print(f"there is no hotel whthin 5000m around {row['City']}")
        final_df.loc[index, "Hotel"] = " "
final_df = final_df.dropna(how = 'any')
final_df.head()

there is no hotel whthin 5000m around Anuchino
there is no hotel whthin 5000m around Sychëvka
there is no hotel whthin 5000m around Shirak
there is no hotel whthin 5000m around Palana
there is no hotel whthin 5000m around Bonfim


,Country,City,Longtitude,Latitude,Temperature,Cloudiness,Humidity,Wind Speed,Hotel
0,BR,São Marcos,-51.0681,-28.9711,283.23,0,88,1.42,Hotel São Marcos
1,BR,Canguçu,-52.6756,-31.3950,282.79,0,82,1.49,Casa Paulista
2,US,Laguna,-121.4238,38.4210,290.57,1,88,0.89,Holiday Inn Express & Suites Elk Grove Central...
3,US,Saint George,-113.5841,37.1041,297.31,1,16,2.24,Best Western Plus Abbey Inn
4,CK,Avarua,-159.7750,-21.2078,296.18,20,64,3.60,Paradise Inn


In [103]:
info_box_template = """
<dl>
<dt>Hotel</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in final_df.iterrows()]
locations = final_df[["Latitude", "Longtitude"]].astype(float)

markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))